# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install --upgrade azure-ai-projects

In [2]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
# Load environment variables from .env file
load_dotenv()

True

## define project client

In [5]:

endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Single agent

In [6]:
agent = project_client.create_agent(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_0D8bIxgLzplCR5Wg7ub41LAg


In [7]:
agent

{'id': 'asst_0D8bIxgLzplCR5Wg7ub41LAg', 'object': 'assistant', 'created_at': 1749587993, 'name': 'my-agent', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are helpful agent', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

In [8]:
prompt = "Hello"

In [9]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")



Created thread, thread ID: thread_4sMpOqpA0zKAUBTw1bdFfzV9


In [10]:
 # [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_UZWdEAZp2nOJqBtAFXlpxN40


In [11]:
thread

{'id': 'thread_4sMpOqpA0zKAUBTw1bdFfzV9', 'object': 'thread', 'created_at': 1749587994, 'metadata': {}, 'tool_resources': {}}

In [12]:
message

{'id': 'msg_UZWdEAZp2nOJqBtAFXlpxN40', 'object': 'thread.message', 'created_at': 1749587994, 'assistant_id': None, 'thread_id': 'thread_4sMpOqpA0zKAUBTw1bdFfzV9', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, tell me a joke', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [13]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_JTFExDAqECbvNSxrqwVFCHtL


In [ ]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

run.usage

Run status: RunStatus.COMPLETED


In [17]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-10 20:39:56+0000
End     : 2025-06-10 20:39:56+0000
Elapsed : 0:00:00  (0.000 seconds)


In [18]:
# [START list_messages]
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")
# [END list_messages]

MessageRole.USER: Hello, tell me a joke
MessageRole.AGENT: Sure! Why don't skeletons fight each other?

They don't have the guts!


## Another agent

In [19]:
# Create a new agent with different instructions

agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a GenAI agent. Always answer in French",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_iKSt7otYT7fGBAXReVXzIFtg


In [20]:
    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_gYO2Fu8Ne3aEDaX7zcoI2B5G


In [21]:
    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content="What is a GenAI model?")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_ok3zrC82mrne6hVO07t3D4YC


In [22]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_h2mYnJryE16K8ZkYyVDeWo19


In [23]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [24]:
run.usage

{'prompt_tokens': 37, 'completion_tokens': 157, 'total_tokens': 194, 'prompt_token_details': {'cached_tokens': 0}}

In [25]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-10 20:40:00+0000
End     : 2025-06-10 20:40:03+0000
Elapsed : 0:00:03  (3.000 seconds)


In [26]:
    # [START list_messages]
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

MessageRole.USER: What is a GenAI model?
MessageRole.AGENT: Un modèle GenAI (ou Generative AI, Intelligence Artificielle Générative) est un système d'intelligence artificielle capable de créer de nouveaux contenus, tels que du texte, des images, de la musique ou même du code informatique, à partir de données d’entraînement. Ces modèles apprennent à partir de grandes quantités de données pour générer des résultats qui ressemblent fortement à ceux produits par des humains. Les modèles GenAI les plus connus sont les modèles de langage, comme GPT (pour le texte), DALL-E (pour les images) ou encore des générateurs de musique. Ils sont utilisés dans de nombreux domaines, notamment la rédaction automatisée, la création artistique, l’assistance à la programmation ou le soutien à la prise de décision.


## Cleaning

In [27]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")


Agent ID: asst_iKSt7otYT7fGBAXReVXzIFtg, Name: my-agent, Model: gpt-4.1, Instructions: You are a GenAI agent. Always answer in French
Agent ID: asst_0D8bIxgLzplCR5Wg7ub41LAg, Name: my-agent, Model: gpt-4.1, Instructions: You are helpful agent


In [29]:

agents = project_client.list_agents()
for agent in agents:
    project_client.delete_agent(agent.id)
print("All agents deleted.")

All agents deleted.


In [30]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")